In [2]:
import pandas as pd
#df=pd.read_csv('https://cloud.minsa.gob.pe/s/AC2adyLkHCKjmfm/download', sep=';', usecols=['FECHA_CORTE', 'EDAD', 'SEXO', 'FECHA_RESULTADO', 'METODODX', 'DEPARTAMENTO'], parse_dates=['FECHA_RESULTADO'])
df=pd.read_csv('positivos_covid.csv', sep=';', usecols=['FECHA_CORTE', 'EDAD', 'SEXO', 'FECHA_RESULTADO', 'METODODX', 'DEPARTAMENTO'], parse_dates=['FECHA_RESULTADO'], dtype={'FECHA_CORTE': str})
fecha_corte=df['FECHA_CORTE'].bfill().ffill().drop_duplicates().set_axis(['fecha_corte'])
#fecha_corte.to_json("../resultados/fecha_corte_positivos.json")
df

/tmp/ipykernel_62653/3054445455.py:3: DtypeWarning: Columns (1,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('positivos_covid.csv', sep=';', usecols=['FECHA_CORTE', 'EDAD', 'SEXO', 'FECHA_RESULTADO', 'METODODX', 'DEPARTAMENTO'], parse_dates=['FECHA_RESULTADO'], dtype={'FECHA_CORTE': str})


,FECHA_CORTE,DEPARTAMENTO,METODODX,EDAD,SEXO,FECHA_RESULTADO
0,20220131,LIMA,PR,37.0,FEMENINO,2020-09-12
1,20220131,LIMA,PR,25.0,MASCULINO,2020-12-17
2,20220131,AREQUIPA,PCR,27.0,MASCULINO,2022-01-20
3,20220131,ICA,PR,20.0,FEMENINO,2020-08-22
4,20220131,ICA,AG,22.0,FEMENINO,2022-01-15
...,...,...,...,...,...,...
3460655,20220131,LIMA,PR,37.0,MASCULINO,2020-09-11
3460656,20220131,CALLAO,PCR,25.0,MASCULINO,2021-04-04
3460657,20220131,LORETO,PCR,15.0,FEMENINO,2022-01-03
3460658,20220131,LIMA,PCR,54.0,FEMENINO,2021-02-20


In [3]:
# DIARIO AG, PCR, PR
df_positivos=df[['FECHA_RESULTADO','METODODX','SEXO']].groupby(['FECHA_RESULTADO','METODODX']).count()
df_positivos=df_positivos.reset_index()
df_positivos=df_positivos.pivot(index='FECHA_RESULTADO', columns='METODODX', values='SEXO')
df_positivos=df_positivos.rename_axis(None, axis=1)
df_positivos=df_positivos.fillna(0).astype('int')
df_positivos

,AG,PCR,PR
FECHA_RESULTADO,,,
2020-03-06,0,1,0
2020-03-07,0,5,0
2020-03-08,0,2,0
2020-03-09,0,3,0
2020-03-10,0,1,0
...,...,...,...
2022-01-26,21324,16577,0
2022-01-27,21944,11376,0
2022-01-28,18893,8160,0


In [8]:
# DIARIO AG, PCR, PR DESDE 2022
df_positivos_2022=df_positivos[df_positivos.index >= '2022-01-01']
df_positivos_2022

,AG,PCR,PR
FECHA_RESULTADO,,,
2022-01-01,1014,1168,0
2022-01-02,1619,1317,195
2022-01-03,6511,2558,285
2022-01-04,9039,3340,2
2022-01-05,11263,3778,0
2022-01-06,14499,4702,0
2022-01-07,14085,4566,0
2022-01-08,16430,6152,1
2022-01-09,7382,3588,0


In [21]:
# ACUMULADO AG, PCR, PR
df_positivos_cum=df_positivos.cumsum()
df_positivos_cum

,AG,PCR,PR
FECHA_RESULTADO,,,
2020-03-06,0,1,0
2020-03-07,0,6,0
2020-03-08,0,8,0
2020-03-09,0,11,0
2020-03-10,0,12,0
...,...,...,...
2022-01-26,1256425,940582,955884
2022-01-27,1278369,951958,955884
2022-01-28,1297262,960118,955884


In [22]:
col_poblacion=[426806,
1180638,
430736,
1497438,
668213,
1453711,
1129854,
1357075,
365317,
760267,
975182,
1361467,
2016771,
1310785,
10628470,
1027559,
173811,
192740,
271904,
2047954,
1237997,
899648,
370974,
251521,
589110]

In [23]:
# ACUMULADO POR DEPARTAMENTO
df_positivos_departamento=df[['DEPARTAMENTO','METODODX']].dropna()
def function(valueX):
    if 'LIMA REGION' in valueX:
      return 'LIMA'
    elif 'ARICA' in valueX:
      return 'TACNA'
    else:
      return valueX
df_positivos_departamento['DEPARTAMENTO']=df_positivos_departamento['DEPARTAMENTO'].map(function)
df_positivos_departamento=df_positivos_departamento.groupby(['DEPARTAMENTO']).count()
df_positivos_departamento['POBLACION']=col_poblacion
df_positivos_departamento['INDICE']=round(df_positivos_departamento['METODODX']/(df_positivos_departamento['POBLACION']/100000)).astype('int')
df_positivos_departamento


,METODODX,POBLACION,INDICE
DEPARTAMENTO,,,
AMAZONAS,39741,426806,9311
ANCASH,114449,1180638,9694
APURIMAC,33962,430736,7885
AREQUIPA,185794,1497438,12407
AYACUCHO,45173,668213,6760
CAJAMARCA,89494,1453711,6156
CALLAO,138787,1129854,12284
CUSCO,109686,1357075,8083
HUANCAVELICA,22197,365317,6076


In [24]:
# ACUMULADO POR GRUPO ETARIO
bins = [18,20,30,40,50,60,70,80,df['EDAD'].max()+1]
labels = ['18 a 19 años','20 a 29 años','30 a 39 años','40 a 49 años','50 a 59 años','60 a 69 años','70 a 79 años','80 años a más']
poblacion_por_grupo_etario = [900000,5700000,5300000,4400000,3300000,2300000,1300000,700000] 
df_positivos_edades = df
df_positivos_edades['GRUPO_ETARIO'] = pd.cut(df['EDAD'], bins=bins, labels=labels, right=False)
df_positivos_edades = df_positivos_edades.groupby(['GRUPO_ETARIO'])["METODODX"].count().reset_index()
df_positivos_edades.rename(columns = {'METODODX':'POSITIVOS'}, inplace = True)
df_positivos_edades['POBLACION']=poblacion_por_grupo_etario
df_positivos_edades['PORCENTAJE']=round(df_positivos_edades['POSITIVOS']/df_positivos_edades['POBLACION']*100,2)
df_positivos_edades=df_positivos_edades.set_index('GRUPO_ETARIO')
df_positivos_edades

,POSITIVOS,POBLACION,PORCENTAJE
GRUPO_ETARIO,,,
18 a 19 años,67423,900000,7.49
20 a 29 años,672840,5700000,11.80
30 a 39 años,762548,5300000,14.39
40 a 49 años,604041,4400000,13.73
50 a 59 años,447110,3300000,13.55
60 a 69 años,280392,2300000,12.19
70 a 79 años,141280,1300000,10.87
80 años a más,70497,700000,10.07


In [9]:
# ACUMULADO POR GRUPO ETARIO - 2022
bins = [5,12,18,30,40,50,60,80,df['EDAD'].max()+1]
labels = ['5 a 11 años','12 a 17 años','18 a 29 años','30 a 39 años','40 a 49 años','50 a 59 años','60 a 79 años','80 años a más']
poblacion_por_grupo_etario = [4201842,3614488,6788969,5382481,4604711,3524112,3851743,812904]
df_positivos_edades_2022 = df[df['FECHA_RESULTADO']>= '2022-01-01']
df_positivos_edades_2022['GRUPO_ETARIO'] = pd.cut(df['EDAD'], bins=bins, labels=labels, right=False)
df_positivos_edades_2022 = df_positivos_edades_2022.groupby(['GRUPO_ETARIO'])["METODODX"].count().reset_index()
df_positivos_edades_2022.rename(columns = {'METODODX':'POSITIVOS'}, inplace = True)
df_positivos_edades_2022['POBLACION']=poblacion_por_grupo_etario
df_positivos_edades_2022['PORCENTAJE']=round(df_positivos_edades_2022['POSITIVOS']/df_positivos_edades_2022['POBLACION']*100,2)
df_positivos_edades_2022=df_positivos_edades_2022.set_index('GRUPO_ETARIO')
df_positivos_edades_2022

/tmp/ipykernel_62653/766422908.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_positivos_edades_2022['GRUPO_ETARIO'] = pd.cut(df['EDAD'], bins=bins, labels=labels, right=False)


,POSITIVOS,POBLACION,PORCENTAJE
GRUPO_ETARIO,,,
5 a 11 años,26690,4201842,0.64
12 a 17 años,30887,3614488,0.85
18 a 29 años,265215,6788969,3.91
30 a 39 años,242063,5382481,4.50
40 a 49 años,168260,4604711,3.65
50 a 59 años,104801,3524112,2.97
60 a 79 años,87436,3851743,2.27
80 años a más,13773,812904,1.69


In [25]:
#TOTAL POSITIVOS
#positivos = pd.Series([df['FECHA_CORTE'].unique()[0], df[['FECHA_CORTE'][0]].count(), df_positivos.tail(1).sum(axis=1)[0]], index=['fecha_corte', 'total_positivos', 'incremento_positivos'])
#positivos.to_json('../resultados/positivos.json')

In [26]:
from requests_html import HTMLSession
session = HTMLSession()
r = session.get('https://www.gob.pe/coronavirus#casos')
scrap_covid_acumulado = r.html.find('.text-base.w-full.tracking-tight')
scrap_covid_diario = r.html.find('.font-bold.text-2xl')
scrap_covid = scrap_covid_acumulado + scrap_covid_diario
d = []
for number in scrap_covid:
    d.append(number.text)
covid = pd.Series(d,index=['total_confirmados', 'total_altas','total_fallecidos','ayer_confirmados','ayer_altas','ayer_fallecidos'])
#covid.to_json('../resultados/covid.json')
covid

total_confirmados    3262165
total_altas            99281
total_fallecidos      205985
ayer_confirmados       13902
ayer_altas               174
ayer_fallecidos           79
dtype: object

In [27]:
# DIARIO AG, PCR, PR: TACNA
df_tacna=df[df['DEPARTAMENTO'] == 'TACNA']
df_positivos_tacna=df_tacna[['FECHA_RESULTADO','METODODX','SEXO']].groupby(['FECHA_RESULTADO','METODODX']).count()
df_positivos_tacna=df_positivos_tacna.reset_index()
df_positivos_tacna=df_positivos_tacna.pivot(index='FECHA_RESULTADO', columns='METODODX', values='SEXO')
df_positivos_tacna=df_positivos_tacna.rename_axis(None, axis=1)
df_positivos_tacna=df_positivos_tacna.fillna(0).astype('int')
df_positivos_tacna

,AG,PCR,PR
FECHA_RESULTADO,,,
2020-03-28,0,1,0
2020-03-31,0,1,0
2020-04-04,0,6,0
2020-04-07,0,0,1
2020-04-14,0,0,3
...,...,...,...
2022-01-26,577,110,0
2022-01-27,683,128,0
2022-01-28,536,154,0
